In [1]:
import functools
import json
import math
import pandas as pd
import pennylane as qml
import pennylane.numpy as np
import scipy

In [2]:
def circuit():
    
    
    #hadamard wire 1
    qml.U3(0, (np.pi)/4, (np.pi)/4, wires=1)
    qml.U3((np.pi)/2, -(np.pi)/2, (np.pi)/2, wires=1)
    qml.U3(0, (np.pi)/4, (np.pi)/4, wires=1)
    
    #cnot wire 0/1
    qml.CNOT(wires=(0,1))
    
    #hadamard wire 1
    qml.U3(0, (np.pi)/4, (np.pi)/4, wires=1)
    qml.U3((np.pi)/2, -(np.pi)/2, (np.pi)/2, wires=1)
    qml.U3(0, (np.pi)/4, (np.pi)/4, wires=1)
    
    #z
    qml.U3(0, (np.pi)/2, (np.pi)/2, wires=0)
    
    #hadamard wire 2
    qml.U3(0, (np.pi)/4, (np.pi)/4, wires=2)
    qml.U3((np.pi)/2, -(np.pi)/2, (np.pi)/2, wires=2)
    qml.U3(0, (np.pi)/4, (np.pi)/4, wires=2)
    

In [3]:
qml.matrix(circuit)()

array([[ 7.07106781e-01-4.32978028e-17j,  7.07106781e-01-1.29893408e-16j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+4.32978028e-17j,  5.30245156e-33+4.32978028e-17j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j],
       [ 7.07106781e-01-4.32978028e-17j, -7.07106781e-01+1.29893408e-16j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+4.32978028e-17j, -5.30245156e-33-4.32978028e-17j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        -7.07106781e-01+1.29893408e-16j, -7.07106781e-01+2.16489014e-16j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         7.85046229e-17+4.32978028e-17j,  7.85046229e-17+4.32978028e-17j],
       [ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        -7.07106781e-01+1.29893408e

In [4]:
# These functions are responsible for testing the solution.

def run(input: str) -> str:
    matrix = qml.matrix(circuit)().real

    with qml.tape.QuantumTape() as tape:
        circuit()

    names = [op.name for op in tape.operations]
    return json.dumps({"matrix": matrix.tolist(), "gates": names})

def check(user_output: str, expected_output: str) -> str:
    parsed_output = json.loads(user_output)
    matrix_user = np.array(parsed_output["matrix"])
    gates = parsed_output["gates"]

    solution = (
        1
        / np.sqrt(2)
        * np.array(
            [
                [1, 1, 0, 0, 0, 0, 0, 0],
                [1, -1, 0, 0, 0, 0, 0, 0],
                [0, 0, -1, -1, 0, 0, 0, 0],
                [0, 0, -1, 1, 0, 0, 0, 0],
                [0, 0, 0, 0, 1, 1, 0, 0],
                [0, 0, 0, 0, 1, -1, 0, 0],
                [0, 0, 0, 0, 0, 0, 1, 1],
                [0, 0, 0, 0, 0, 0, 1, -1],
            ]
        )
    )

    assert np.allclose(matrix_user, solution)
    assert len(set(gates)) == 2 and "U3" in gates and "CNOT" in gates

In [5]:
test_cases = [['No input', 'No output']]

In [6]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input 'No input'...
Correct!
